In [ ]:
import matplotlib
import matplotlib.pyplot as plt

from turtle import color
from ray.tune import ExperimentAnalysis
from matplotlib.ticker import MaxNLocator

import pandas as pd
from torch import tensor
def plot(show_ECE=0, specific="", experiment_name="a1", save = False, save_path = "plot.svg", inds = None, lim = None):
    analysis = ExperimentAnalysis("./results/"+experiment_name)
    ax = None
    dfs = analysis.trial_dataframes
    for k, d in list(dfs.items())[:]:
        if specific != "":
            if not specific in k: continue
        if "v01" in k:
            if show_ECE == 1:
                if lim:
                    ax = d.ece[:lim].plot(ax=ax, legend=False, color="red")
                else:
                    ax = d.ece.plot(ax=ax, legend=False, color="red")
            elif show_ECE == 0:
                if lim:
                    ax = d.accuracy[:lim].plot(ax=ax, legend=False, color="red")
                else:
                    ax = d.accuracy.plot(ax=ax, legend=False, color="red")
            else:
                if lim:
                    ax = pd.Series([float(eval(i)) for i in d.example_loss])[:lim].plot(ax=ax, legend=False, color="red")
                else:
                    ax = pd.Series([float(eval(i)) for i in d.example_loss]).plot(ax=ax, legend=False, color="red")
        else:
            if show_ECE == 1:
                if lim:
                    ax = d.ece[:lim].plot(ax=ax, legend=False, color="blue")
                else:
                    ax = d.ece.plot(ax=ax, legend=False, color="blue")
            elif show_ECE == 0:
                if lim:
                    ax = d.accuracy[:lim].plot(ax=ax, legend=False, color="blue")
                else:
                    ax = d.accuracy.plot(ax=ax, legend=False, color="blue")
            else:
                if lim:
                    ax = pd.Series([float(eval(i)) for i in d.example_loss])[:lim].plot(ax=ax, legend=False, color="blue")
                else:
                    ax = pd.Series([float(eval(i)) for i in d.example_loss]).plot(ax=ax, legend=False, color="blue")

    ax.set_xlabel("Epochs")
    if show_ECE == 1:
        ax.set_ylabel("ECE")
    elif show_ECE == 0:
        ax.set_ylabel("Accuracy")
    else:
        ax.set_ylabel("Example Loss")
    # set x axis to be integers
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    # set y-axis values to 2 decimal places
    ax.yaxis.set_major_formatter('{x:.2f}')
    # adjust bottom to avoid cutoff
    plt.subplots_adjust(bottom=0.15)

    # adjust left to avoid cutoff
    plt.subplots_adjust(left=0.15)

    plt.legend(["IS-loss", "Cross-Entropy"])
    if save:
        if show_ECE == 1:
            plt.savefig(save_path.replace(".svg", "_ece.svg"))
        elif show_ECE == 0:
            plt.savefig(save_path.replace(".svg", "_acc.svg"))
        plt.savefig(save_path, format="svg")
    
ece = 0
lim = 100
plot(
    show_ECE=ece, # 1 for ECE, 0 for accuracy, 2 for loss_example
    specific="e2ea5_00000", # if you want to plot a specific experiment, put the name here else leave blank
    experiment_name="a18",
    save=True,
    lim=lim,
)

plot(
    show_ECE=ece, # 1 for ECE, 0 for accuracy, 2 for loss_example
    specific="e2ea5_00003", # if you want to plot a specific experiment, put the name here else leave blank
    experiment_name="a18",
    save=True,
    lim=lim,
)

In [ ]:
experiment = "a18"
from ray.tune import ExperimentAnalysis
a = ExperimentAnalysis("./results/" + experiment)
dfs = a.trial_dataframes
dfs = pd.DataFrame([dfs[list(dfs.keys())[i]].iloc[99] for i in range(4)])
#dfs = a.dataframe()
dfs = dfs.drop(columns=['timesteps_total','episodes_total', 'experiment_id','date',#'logdir','config/batch_size',
'timestamp', 'pid', 'hostname', 'node_ip','time_since_restore', 'timesteps_since_restore', 
'warmup_time', 'example_loss', 'training_iteration','iterations_since_restore', 'done', 
'time_this_iter_s', 'mixed-score','freqs','time_total_s',#'config/criterion', 
])#'config/eps', 'config/lr', 'config/lr','iterations_since_restore','trial_id', 'config/model'
dfs#dfs.sort_values("accuracy", ascending=False)

In [ ]:
from reliabilityplot import plot_reliability
experiment = "a18"
from ray.tune import ExperimentAnalysis
dfs = ExperimentAnalysis("./results/" + experiment).trial_dataframes
freqs = dfs[list(dfs.keys())[0]].iloc[99]["freqs"]
nan = float("nan")
freqs = eval("torch."+(freqs))
plot_reliability(freqs)

In [ ]:
import torch
from reliabilityplot import plot_reliability
from ray.tune import ExperimentAnalysis
dfs = ExperimentAnalysis("./results/a12").dataframe()
nan = float("nan")
freqs = dfs["freqs"][4]
freqs = eval("torch."+(freqs))#freqs = eval("torch."+(dfs["freqs"][dfs[dfs["trial_id"]=="01e8f_00003"].index[0]]))
plot_reliability(freqs)

In [97]:
import library
import tune

import importlib
importlib.reload(library)
importlib.reload(tune)
import torch
import torch.nn as nn
import torchvision.models as models

device = "cuda"
model = models.resnet18(num_classes=10).to(device)
optim = torch.optim.Adam(model.parameters(), 0.01)
train_loader, test_loader = library.load_data(250)
for i in range(2):
    l = tune.train(model, optim, library.itakura_saito_loss_v01,
    train_loader, device)
    #print(l, tune.test(model, test_loader, device, ece=True, n_bins=10)[:2])

Files already downloaded and verified


In [108]:
#measuring the time it takes to execute IS-loss vs Cross-Entropy
import timeit
a = [(data, target) for batch_idx, (data, target) in enumerate(train_loader)][0]
data = a[0].to("cuda")
target = a[1].to("cuda")
model = model.to("cuda")
model.train()
out = model(data)
ce = nn.CrossEntropyLoss()
isloss = library.itakura_saito_loss_v01
print(timeit.timeit("library.itakura_saito_loss_v01(out, target, 0.001)", globals=globals(), number=100000))
print(timeit.timeit("isloss(out, target, 0.001)", globals=globals(), number=100000))
print(timeit.timeit("nn.CrossEntropyLoss()(out, target)", globals=globals(), number=100000))
print(timeit.timeit("ce(out, target)", globals=globals(), number=100000))

11.053031113930047
10.88397036679089
5.176125489873812
2.2806896639522165


In [ ]:
import importlib
importlib.reload(library)
from reliabilityplot import plot_reliability
acc, ece, freqs = library.test(model, test_loader, device, ece=True)
print(acc, ece)
plot_reliability(freqs)

In [ ]:
import library
import tune

import torch
import torch.nn as nn
import torchvision.models as models

device = "cuda"
model2 = models.resnet152(num_classes=10).to(device)
train_loader, test_loader = library.load_data(250)
for i in range(2):
    tune.train(model2, torch.optim.Adam(model2.parameters(), 0.001), nn.CrossEntropyLoss(), train_loader, device)

In [ ]:
import importlib
importlib.reload(library)
acc, ece, freqs = library.test(model2, test_loader, device, ece=True)
print(acc, ece)
plot_reliability(freqs)

In [ ]:
from ray.tune import ExperimentAnalysis
dfs = ExperimentAnalysis("./results/a16")
dfs.get_best_config(mode="max", metric="accuracy", scope="last-5-avg")